In [20]:
# Import packages
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import iaml01cw2_helpers as hp
from scipy.spatial import distance
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, log_loss, mean_squared_error
KNeighboursClassifier = KNeighborsClassifier # For the Brits!
%matplotlib inline

In [2]:
Xtrn, Ytrn, Xtst, Ytst = hp.load_FashionMNIST('D:\iaml-cw2\data\data')
Xtrn=Xtrn/255
Xtst=Xtst/255

In [3]:
Xtrn_orig=Xtrn.copy()
Xtst_orig=Xtst.copy()

In [4]:
#1.2
df=pd.DataFrame(Xtrn)
dftst=pd.DataFrame(Xtst)
Xmean=np.mean(Xtrn,axis=0)
Xtrn_nm=df.transpose().apply(lambda x:x-Xmean).transpose()
Xtst_nm=dftst.transpose().apply(lambda x:x-Xmean).transpose()
dfy=pd.DataFrame(Ytrn,columns=['y'])
frames=[df,dfy]
dft=pd.concat(frames,axis=1)
fig,axs = plt.subplots(10,5,figsize=(12,17))
for i in range (10):
    c=dft[dft['y']==i].drop(columns=['y'])
    m=c.mean()
    r=c.shape[0]
    d=[0 for x in range (r)]
    for j in range (r):
        d[j]=distance.euclidean(m,c.loc[c.index[j]])
    s=sorted(zip(d,c.index),key=lambda t:t[0])
    axs[i][0].imshow(np.array(m).reshape(28,28),cmap='gray_r')
    axs[i][0].set(ylabel='Class '+str(i)+'-Mean')
    closest=np.array(c.loc[s[0][1]]).reshape(28,28)
    second_closest=np.array(c.loc[s[1][1]]).reshape(28,28)
    second_furtherest=np.array(c.loc[s[-2][1]]).reshape(28,28)
    furtherest=np.array(c.loc[s[-1][1]]).reshape(28,28)
    axs[i][1].imshow(np.array(closest).reshape(28,28),cmap='gray_r')
    axs[i][1].set(ylabel='Class '+str(i)+'-'+str(s[0][1]))
    axs[i][2].imshow(np.array(second_closest).reshape(28,28),cmap='gray_r')
    axs[i][2].set(ylabel='Class '+str(i)+'-'+str(s[1][1]))
    axs[i][3].imshow(np.array(second_furtherest).reshape(28,28),cmap='gray_r')
    axs[i][3].set(ylabel='Class '+str(i)+'-'+str(s[-2][1]))
    axs[i][4].imshow(np.array(furtherest).reshape(28,28),cmap='gray_r')
    axs[i][4].set(ylabel='Class '+str(i)+'-'+str(s[-1][1]))
plt.savefig('1-2.png')

In [1]:
#1.1
np.array(Xtrn_nm)[0][0:4]

In [2]:
#1.1
np.array(Xtst_nm)[-1][0:4]

In [4]:
#1.5
pca=PCA(10)
pca.fit(Xtrn_nm)
a=pca.explained_variance_ratio_
for i in [1,2,3,4,5,6,7,8,9]:
    a[i]=a[i-1]+a[i]

In [ ]:
#1.3
pca=PCA(5)
pca.fit(Xtrn_nm)
r=pca.explained_variance_

In [14]:
#1.4
r=pca.explained_variance_ratio_
sr=[0,0,0,0,0]
for i in range (5):
    if(i==0):
        sr[i]=r[i]
    else:
        sr[i]=sr[i-1]+r[i]
plt.plot(sr)
plt.xticks([i for i in range (5)],labels=[i+1 for i in range (5)])
plt.title('Cumulative Explained Variance Ratio')
plt.xlabel('Component no.')
plt.ylabel('Cumulative Ratio')

In [9]:
#1.5
pca=PCA(10)
pca.fit(Xtrn_nm)
fig,axs=plt.subplots(2,5,figsize=(12,5))
for i in range (10):
    a=i//5
    b=i%5
    axs[a][b].imshow(np.array(pca.components_[i]).reshape(28,28),cmap='gray_r')
    axs[a][b].set_title('Component '+str(i+1))

In [8]:
#1.6,1.7
dfy=pd.DataFrame(Ytrn,columns=['y'])
dfnm=pd.DataFrame(Xtrn_nm)
frames=[dfnm,dfy]
dftnm=pd.concat(frames,axis=1)
a=[5,20,50,200]
rmse_all=[[0 for i in range (4)]for j in range (10)]
fig,axs=plt.subplots(10,4,figsize=(10,20))
for i in range (10):
    c=dftnm[dftnm['y']==i].drop(columns=['y'])
    m=c.mean()
    first=c.loc[c.index[0]]
    for j in range (4):
        pca=PCA(a[j])
        pca.fit(c)
        re=pca.inverse_transform(first.dot(pca.components_.transpose()))
        rmse=np.sqrt(mean_squared_error(first,re))
        rmse_all[i][j]=rmse
        re=re+m
        axs[i][j].imshow(np.array(re).reshape(28,28),cmap='gray_r')
        axs[i][j].set(ylabel='Class '+str(i)+',K='+str(a[j]))
plt.savefig('1-7.png')
print(rmse_all)

In [21]:
#1.8
pca=PCA(2)
pca.fit(Xtrn_nm,Ytrn)
dftnmx=dftnm.drop(columns=['y'])
c=dftnmx.dot(pca.components_.transpose())
frames=[c,dfy]
dftnm2=pd.concat(frames,axis=1)
plt.scatter(dftnm2[0],dftnm2[1],c=Ytrn,s=1,cmap='coolwarm')
plt.title('Training samples in 2-D')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
cb=plt.colorbar(aspect=15)
cb.set_label('Color for each Class')
plt.savefig('1-8.png')

In [6]:
#2.1
lg=LogisticRegression()
lg.fit(Xtrn_nm,Ytrn)
lg.score(Xtst_nm,Ytst)

In [ ]:
#2.1
cm=confusion_matrix(Ytst,lg.predict(Xtst_nm))

In [7]:
#2.2
svc=SVC()
svc.fit(Xtrn_nm,Ytrn)
svc.score(Xtst_nm,Ytst)

In [9]:
#2.2
cm=confusion_matrix(Ytst,svc.predict(Xtst_nm))

In [10]:
#2.3
pca=PCA(2)
pca.fit(Xtrn_nm,Ytrn)
sigma1=np.sqrt(pca.explained_variance_[0])
sigma2=np.sqrt(pca.explained_variance_[1])
x=np.linspace(-5*sigma1,5*sigma1,100)
y=np.linspace(-5*sigma2,5*sigma2,100)
X,Y=np.meshgrid(x,y)
Xtst2d=Xtst.dot(pca.components_.transpose())
z=[[lg.predict(pca.inverse_transform([[x[i],y[j]]]))[0] for i in range (100)]for j in range (100)]
plt.contourf(X,Y,z,levels=np.arange(-1,9),cmap='coolwarm')
plt.title('Decision Regions for Logistic Regression')
plt.xlabel('Sigma1')
plt.ylabel('Sigma2')
cb=plt.colorbar(aspect=15)
cb.set_label('Color for each Class')
cb.set_ticks(np.arange(-0.5,8.5))
cb.set_ticklabels(np.arange(0,9))
plt.savefig('2-3.png')

In [21]:
#2.4
zs=[[svc.predict(pca.inverse_transform([[x[i],y[j]]]))[0] for i in range (100)]for j in range (100)]

In [11]:
#2.5
frames=[dft[dft['y']==i].head(1000) for i in range(10)]
small=pd.concat(frames,axis=0)
Xsmall=small.drop(columns=['y'])
Ysmall=small['y']
kf=KFold(3)
x=np.linspace(-2,3,10)
a=0
m=[0 for i in range (10)]
for i in range (10):
    svc=SVC(10**x[i])
    m[i]=cross_val_score(svc,Xsmall,Ysmall,cv=3)

In [12]:
#2.4
plt.contourf(X,Y,zs,levels=np.arange(-1,10),cmap='coolwarm')
plt.title('Decision Regions for SVC')
plt.xlabel('Sigma1')
plt.ylabel('Sigma2')
cb=plt.colorbar()
cb.set_label('Color for each Class')
cb.set_ticks(np.arange(-0.5,9.5))
cb.set_ticklabels(np.arange(0,10))
plt.savefig('2-4.png')

In [13]:
#2.5
mm=[np.mean(m[i]) for i in range (10)]
plt.plot(x,mm)
plt.title('Relationship between Accuracy and C')
plt.xlabel('Regularisation Parameter(log-scale)')
plt.ylabel('Mean Accuracy')
plt.savefig('2-5.png')

In [16]:
#2.5
s=sorted(zip(x,mm),key=lambda t:t[1])

In [17]:
#2.6
svc2=SVC(10**x[6])
svc2.fit(Xtrn_nm,Ytrn)

In [18]:
#2.6
svc2.score(Xtrn_nm,Ytrn)

In [19]:
#2.6
svc2.score(Xtst_nm,Ytst)